In [104]:
import os
import time
import random
import gc

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn

from albumentations.pytorch import ToTensorV2

from torch import optim
from torch.optim import Adam
from torch.optim import optimizer
from torch.optim import lr_scheduler

from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torch.optim.lr_scheduler import CosineAnnealingLR
from IPython.display import display

from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix


import matplotlib.pyplot as plt
import seaborn as sns

import wandb


def set_seed(seed_value):
    """모든 랜덤 시드를 고정합니다."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

In [105]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# PATH
MODEL_PATH = '/upstage-cv-classification-cv2/best_model.pth'
MODEL_NAME = 'efficientnet_b4'

TRAIN_AUG_CSV_PATH = '/upstage-cv-classification-cv2/data/train_translate_aug.csv'
TRAIN_SAMPLE_CSV_PATH = '/upstage-cv-classification-cv2/data/train_sample.csv'
TRAIN_AUG_IMAGE_PATH = '/upstage-cv-classification-cv2/data/train_translate_aug'

VALID_CSV_PATH = '/upstage-cv-classification-cv2/data/valid.csv'
VALID_SAMPLE_CSV_PATH = '/upstage-cv-classification-cv2/data/valid_sample.csv'
VALID_IMAGE_PATH = '/upstage-cv-classification-cv2/data/valid'

TEST_CSV_PATH = '/upstage-cv-classification-cv2/data/sample_submission.csv'
TEST_IMAGE_PATH = '/upstage-cv-classification-cv2/data/test'

RESULT_CSV_PATH = '/upstage-cv-classification-cv2'

WANDB_PROJECT_NAME = 'model_enhance_37'

# sampling

In [106]:
train_df = pd.read_csv(TRAIN_AUG_CSV_PATH)

train_3_df = train_df[train_df['target'] == 3]
train_7_df = train_df[train_df['target'] == 7].sample(n = 2700, random_state=42)
train_other_df = train_df[(train_df['target'] != 3) & (train_df['target'] != 7)].sample(n = 2700, random_state = 42)

train_df = pd.concat([train_3_df, train_7_df, train_other_df], axis=0)
train_df['target'] = train_df['target'].map(lambda x : 1 if x == 3 else 0)
train_df.to_csv(TRAIN_SAMPLE_CSV_PATH, index=False)

In [107]:
valid_df = pd.read_csv(VALID_CSV_PATH)

valid_3_df = valid_df[valid_df['target'] == 3]
valid_7_df = valid_df[valid_df['target'] == 7].sample(n = 10, random_state=42)
valid_other_df = valid_df[(valid_df['target'] != 3) & (valid_df['target'] != 7)].sample(n = 10, random_state = 42)

valid_df = pd.concat([valid_3_df, valid_7_df, valid_other_df], axis=0)
valid_df['target'] = valid_df['target'].map(lambda x : 1 if x == 3 else 0)
valid_df.to_csv(VALID_SAMPLE_CSV_PATH, index=False)

# HyperParameter

In [108]:
# training config
img_size = 380
LR = 1e-3
EPOCHS = 100
BATCH_SIZE = 32
num_workers = 0

patience = 5
min_delta = 0.001 # 성능 개선의 최소 변화량

# Data load

In [109]:
# test image 변환
data_transform = A.Compose([
    A.Resize(height = img_size, width = img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]),
    ToTensorV2()
])

class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image = img)['image']
    
        return img, target

    def get_labels(self):
        return self.df[:, 1] 

trn_dataset = ImageDataset(
    TRAIN_SAMPLE_CSV_PATH,
    TRAIN_AUG_IMAGE_PATH,
    transform = data_transform
)

val_dataset = ImageDataset(
    VALID_SAMPLE_CSV_PATH,
    VALID_IMAGE_PATH,
    transform = data_transform
)

tst_dataset = ImageDataset(
    TEST_CSV_PATH,
    TEST_IMAGE_PATH,
    transform = data_transform
)

labels = trn_dataset.get_labels()
labels = labels.astype(int)

# DataLoader
trn_loader = DataLoader(
    trn_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = num_workers,
    pin_memory = True,
    drop_last = False
)

val_loader = DataLoader(
    val_dataset,
    batch_size = BATCH_SIZE,
    num_workers = 0,
    pin_memory = True,
    drop_last = False
)

tst_loader = DataLoader(
    tst_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = 0,
    pin_memory = True
)

print(len(trn_dataset), len(val_dataset), len(tst_dataset))

10800 32 3140


# Model Train

In [114]:
torch.cuda.empty_cache()
gc.collect()

# model
model = timm.create_model('efficientnet_b4',
                        pretrained=True,
                        num_classes = 1).to(device)

loss_fn = nn.BCEWithLogitsLoss()

optimizer = Adam(model.parameters(), lr = LR)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnet_b4.ra2_in1k)


INFO:timm.models._hub:[timm/efficientnet_b4.ra2_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Missing keys (classifier.weight, classifier.bias) discovered while loading pretrained weights. This is expected if model is being adapted.


In [115]:
def valid_one_epoch(loader, model, loss_fn, device, epoch):
    model.eval()
    valid_loss = 0

    preds_list =[]
    targets_list = []

    with torch.no_grad():
        pbar = tqdm(loader)
        for step, (image, targets) in enumerate(pbar):
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)
       
            valid_loss += loss.item()
        
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

            pbar.set_description(f"Loss: {loss.item():.4f}")

            wandb.log({
                "valid_step" : epoch * len(loader) + step,
                "valid_loss_step" : loss.item()
            })

    valid_loss /= len(loader)
    valid_acc = accuracy_score(targets_list, preds_list)
    valid_f1 = f1_score(targets_list, preds_list, average = 'macro')

    ret = {
        "epoch" : epoch,
        "valid_loss" : valid_loss,
        "valid_acc" : valid_acc,
        "valid_f1" : valid_f1
    }

    wandb.log({
        "valid_epoch" : epoch,
        "val_loss_epoch" : valid_loss,
        "val_acc" : valid_acc,
        "val_f1" : valid_f1
    })

    return ret

In [116]:
# one epoch 학습
def train_one_epoch(train_loader, model, optimizer, loss_fn, device, epoch):
    model.train()
    train_loss = 0
    preds_list =[]
    targets_list = []

    pbar = tqdm(train_loader)
    for step, (image, targets) in enumerate(pbar):
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

        wandb.log({
            "train_step" : epoch * len(train_loader) + step,
            "train_loss_step" : loss.item()
        })
        
    train_loss /= len(train_loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average = 'macro')

    ret = {
        "model" : model,
        "train_epoch" : epoch,
        "train_loss" : train_loss,
        "tarin_acc" : train_acc,
        "train_f1" : train_f1
    }

    wandb.log({
        "train_epoch" : epoch,
        "train_loss_epoch" : train_loss,
        "train_acc" : train_acc,
        "train_f1" : train_f1
    })

    return ret

In [117]:
os.environ['WANDB_SILENT'] = 'true'

f1_scores = []
valid_losses = []
trained_models = []
patience_counter = 0
best_loss = 1


wandb.init(project=WANDB_PROJECT_NAME, name="effi_b4")

for epoch in range(EPOCHS):
    print(f"{epoch} epoch")
    trn_ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device, epoch)
    val_ret = valid_one_epoch(val_loader, model, loss_fn, device, epoch)

    val_loss = val_ret['valid_loss']
    val_f1 = val_ret['valid_f1']

    scheduler.step(val_loss)

    print(f"val loss : {val_loss}")
    print(f"val f1 : {val_f1}")

    f1_scores.append(val_f1)
    valid_losses.append(val_loss)
    trained_models.append(model)

    # 성능 향상
    if val_loss < best_loss - 0.001:
        best_loss = val_loss
        patience_counter = 0
    # 성능 하락
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"Early Stop at {epoch}!!\nbest_loss : {best_loss}")
        break
        
best_model_idx = np.argmax(np.array(f1_scores))
best_model = trained_models[best_model_idx]

wandb.finish()

0 epoch


  0%|          | 0/338 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 636.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 251.75 MiB is free. Process 741947 has 23.44 GiB memory in use. Of the allocated memory 22.40 GiB is allocated by PyTorch, and 736.50 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Model Enhence

In [62]:
pretrained_model = timm.create_model(MODEL_NAME, pretrained=False, num_classes=17).to(device)

# 모델 가중치 로드
pretrained_model.load_state_dict(torch.load(MODEL_PATH))

val_dataset = ImageDataset(
    VALID_CSV_PATH,
    VALID_IMAGE_PATH,
    transform = data_transform
)

val_loader = DataLoader(
    val_dataset,
    batch_size = BATCH_SIZE,
    num_workers = 0,
    pin_memory = True,
    drop_last = False
)

/tmp/ipykernel_347639/2054509229.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_model.load_state_dict(torch.load(MODEL_PATH))


In [65]:
preds_list = []
targets_list = []
is3_preds_list = []

ids = []

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr = LR)

model.eval()

with torch.no_grad():
    pbar = tqdm(val_loader)
    for step, (image, targets) in enumerate(pbar):
        image = image.to(device)
        targets = targets.to(device)

        preds = pretrained_model(image)
        is3_preds = model(image)
        loss = loss_fn(preds, targets)
    
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        is3_preds_list.extend(is3_preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")


Loss: 0.0006: 100%|██████████| 10/10 [00:02<00:00,  4.19it/s]


In [67]:
result_df = pd.DataFrame()
# result_df['id'] = ids
result_df['pred'] = preds_list
result_df['is3_pred'] = is3_preds_list
result_df['target'] = targets_list

result_df[(result_df['pred'] == 3) | (result_df['target'] == 3)]

,pred,is3_pred,target
23,3,0,3
26,3,0,10
44,3,0,7
51,7,0,3
61,3,1,3
87,3,0,4
91,3,1,3
105,7,0,3
136,7,0,3
142,14,0,3


In [68]:
print("Classification Report:")
report = classification_report(targets_list, preds_list, target_names=[str(i) for i in range(17)])
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        20
           3       0.43      0.50      0.46        12
           4       0.76      0.94      0.84        17
           5       0.94      1.00      0.97        16
           6       0.96      0.93      0.95        29
           7       0.60      0.60      0.60        20
           8       0.94      0.94      0.94        16
           9       1.00      1.00      1.00        12
          10       0.95      0.90      0.92        20
          11       0.92      0.92      0.92        13
          12       1.00      0.91      0.95        22
          13       1.00      0.82      0.90        17
          14       0.85      0.79      0.81        14
          15       0.95      1.00      0.97        19
          16       1.00      1.00      1.00        16

   